In [71]:
from datasets import load_dataset
data = load_dataset("bigbio/an_em",name="an_em_bigbio_kb",trust_remote_code=True)

In [72]:
df = data["train"].to_pandas()

In [76]:
try:    
    entities = df[df["entities"].apply(len) != 0]["entities"].tolist()
    entities = [item for sublist in entities for item in sublist]
    new_entites = {}
    for entity in entities: new_entites[entity["id"]] = f"{entity['type']} - {entity['text'][0]}"
    for entity in set([new_entites[entity] for entity in new_entites.keys()]): print(entity)
except:
    pass

Multi-tissue_structure - gastric mucosa
Multi-tissue_structure - inner retinal
Multi-tissue_structure - visual cortex
Pathological_formation - Kaposi Sarcoma
Cell - discohesive tumor cells
Cell - Mast cell
Multi-tissue_structure - atrial
Multi-tissue_structure - artery
Organism_substance - Serum
Cellular_component - membrane
Immaterial_anatomical_entity - orbital
Anatomical_system - Cardiovascular
Organism_subdivision - spinal column
Organism_subdivision - pelvic limb
Organ - cardiac
Cell - RP
Organism_subdivision - palmar
Cell - yolk platelets
Cell - ovarian carcinoma cell lines
Organism_subdivision - vegetable
Tissue - inner nuclear layer
Multi-tissue_structure - neural
Pathological_formation - neonatal ventral hippocampal lesions
Cell - platelets
Multi-tissue_structure - alveolar septa
Anatomical_system - Central nervous system
Multi-tissue_structure - flap
Multi-tissue_structure - cornea
Multi-tissue_structure - temporal lobe
Organism_subdivision - nuts
Organ - heart
Organism_subst

In [77]:
try:
    relations = df[df["relations"].apply(len) != 0]["relations"].tolist()
    relations = [item for sublist in relations for item in sublist]
    for relation in set([f"{new_entites[relation['arg1_id']]} {relation['type'].upper()} {new_entites[relation['arg2_id']]}" for relation in relations]):
        print(relation)
except:
    pass

Multi-tissue_structure - supplementary FRAG Multi-tissue_structure - parietal eyefields
Organ - vascular FRAG Organ - non-vascular smooth muscle
Multi-tissue_structure - slices PART-OF Organ - liver
Tissue - embryonic FRAG Tissue - extraembryonic tissues
Multi-tissue_structure - ventral FRAG Multi-tissue_structure - dorsal striatum
Pathological_formation - primary FRAG Pathological_formation - metastatic carcinomas
Tissue - thyroid FRAG Tissue - eye muscle membranes
Pathological_formation - fragment PART-OF Pathological_formation - tumour
Multi-tissue_structure - left FRAG Multi-tissue_structure - right insula
Tissue - eye muscle FRAG Tissue - thyroid membranes
Pathological_formation - metastatic carcinomas PART-OF Organ - liver
Tissue - visceral endodermal FRAG Tissue - ectodermal layers
Cell - slices PART-OF Cell - cells
Organ - liver PART-OF Developing_anatomical_structure - fetal
Multi-tissue_structure - internal thoracic FRAG Multi-tissue_structure - radial artery
Multi-tissue_str

In [75]:
from huggingface_hub import list_datasets
for dataset in list_datasets():
    if dataset.author == "bigbio":
        print(dataset.id)

bigbio/scitail
bigbio/biosses
bigbio/mednli
bigbio/gad
bigbio/bioasq_task_b
bigbio/blurb
bigbio/an_em
bigbio/anat_em
bigbio/ask_a_patient
bigbio/bc5cdr
bigbio/bc7_litcovid
bigbio/bio_sim_verb
bigbio/bio_simlex
bigbio/bioasq_2021_mesinesp
bigbio/bioasq_task_c_2017
bigbio/bioinfer
bigbio/biology_how_why_corpus
bigbio/biomrc
bigbio/bionlp_shared_task_2009
bigbio/bionlp_st_2011_epi
bigbio/bionlp_st_2011_ge
bigbio/bionlp_st_2011_id
bigbio/bionlp_st_2011_rel
bigbio/bionlp_st_2013_cg
bigbio/bionlp_st_2013_ge
bigbio/bionlp_st_2013_gro
bigbio/bionlp_st_2013_pc
bigbio/bionlp_st_2019_bb
bigbio/biored
bigbio/biorelex
bigbio/bioscope
bigbio/cantemist
bigbio/cas
bigbio/cellfinder
bigbio/chebi_nactem
bigbio/chemdner
bigbio/chemprot
bigbio/chia
bigbio/citation_gia_test_collection
bigbio/codiesp
bigbio/ctebmsp
bigbio/ddi_corpus
bigbio/distemist
bigbio/ebm_pico
bigbio/ehr_rel
bigbio/essai
bigbio/euadr
bigbio/evidence_inference
bigbio/genetag
bigbio/genia_ptm_event_corpus
bigbio/genia_relation_corpus
big